In [1]:
import pandas as pd
import csv
from bs4 import BeautifulSoup
import os

In [2]:
file = "lexile/raw/book-reviews/1984.html"
with open(file, 'r') as f: 
        soup = BeautifulSoup(f.read(), 'html.parser')

## Get All Data

In [65]:
title = soup.h1.text
description = soup.find(class_='field field-name-field-one-liner field-type-text field-label-hidden').get_text()
age = soup.find(class_="csm-green-age").get_text().replace("age ", "")
plot_sum = soup.find(class_="field field-name-field-what-is-story field-type-text-long field-label-hidden").get_text()
csm_rating = soup.find(class_="field field-name-field-any-good field-type-text-long field-label-hidden").get_text()[:-1]

### Parent's rating

In [76]:
def parents_rating(soup): 
    for s in soup.find_all(class_='user-review-statistics adult'):
        adult_rating = s.find("div", attrs={'class': 'stat-wrapper age'})
    return adult_rating.text.replace("age ", "")[:-1])

### Children's rating

In [74]:
def kids_rating(soup):
    for s in soup.find_all(class_='user-review-statistics child'):
        childs_rating = s.find("div", attrs={'class': 'stat-wrapper age'})
    return childs_rating.text.replace("age ", "")[:-1]

In [79]:
a = soup.find(class_="field field-name-field-parents-need-to-know field-type-text-long field-label-hidden").get_text()
a = a.repl

'Parents need to know that 1984 presents an unblinking portrait of life lived under constant surveillance and stands as one of the great dystopian satires of the 20th century. Author George Orwell also wrote Animal Farm, a satirical allegory about the abuse of power.\xa0\xa0\n'

## Get All Data from the Summary box

In [68]:
summary = soup.find(class_="shutter-summary-pane panel-pane pane-product-details")
links = summary.find_all('li')
data = []
d = {}
for link in links: 
    k, v = link.text.split(":")
    k = k.strip()
    v = v.strip()
    d[k] = v
data.append(d)
data

[{'Author': 'George Orwell',
  'Genre': 'Literary Fiction',
  'Topics': 'History',
  'Book type': 'Fiction',
  'Publisher': 'Plume',
  'Publication date': 'June 6, 1949',
  'Number of pages': '368',
  'Last updated': 'January 15, 2019'}]

In [6]:
d

{'Author': 'George Orwell',
 'Genre': 'Literary Fiction',
 'Topics': 'History',
 'Book type': 'Fiction',
 'Publisher': 'Plume',
 'Publication date': 'June 6, 1949',
 'Number of pages': '368',
 'Last updated': 'January 15, 2019'}